<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/DB_lecture_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Consensus**


**Consensus algorithms** in distributed systems allow multiple processes to reach an agreement on a value. 

**FLP impossibility** shows that it is impossible to guarantee consensus in a completely asynchronous system in a bounded time. Even if message delivery is guaranteed, it is impossible for one process to know whether the other one has crashed or is running slowly.

Consensus algorithms assume an asynchronous model and guarantee safety, while an external failure detector can provide information about other processes. Since failure detection is not always fully accurate, there will be situations when a consensus algorithm waits for a process failure to be detected, or when the algorithm is restarted because some process is incorrectly suspected to be faulty.

Processes have to agree on some value proposed by one of the participants, even if some of them happen to crash. A process is said to be **correct** if hasn’t crashed and continues executing algorithm steps. 

Consensus is extremely useful for putting events in a particular order, and ensuring consistency among the participants. Using consensus, we can have a system where processes move from one value to the next one without losing certainty about which values the clients observe.


From a theoretical perspective, consensus algorithms have three properties:
* **Agreement** The decision value is the same for all correct processes.
* **Validity** The decided value was proposed by one of the processes.
* **Termination** All correct processes eventually reach the decision.

**Broadcast**


A broadcast is a communication abstraction often used in distributed systems. **Broadcast algorithms** are used to disseminate information among a set of processes. 

There exist many broadcast algorithms, making different assumptions and providing different guarantees. Broadcast is an important primitive and is used in many places, including consensus algorithms. 

Broadcasts are often used for database replication when a single coordinator node has to distribute the data to all other participants. Making this process reliable is not a trivial matter: if the coordinator crashes after distributing the message to some nodes but not the other ones, it leaves the system in an inconsistent state.

The simplest and the most straightforward way to broadcast messages is through a
**best effort broadcast**. In this case, the sender is responsible for ensuring
message delivery to all the targets. If it fails, the other participants do not try to rebroadcast the message, and in the case of coordinator crash, this type of broadcast will fail silently.


The most straightforward fallback mechanism is to allow every process that received the message to forward it to every other process it’s aware of. 

(+) When the source process fails, other processes detect the failure and continue broadcasting the message, messages still are picked up and delivered by the rest of the system, improving its reliability, and allowing all receivers to see the same messages.

(-) One of the downsides of this approach is the fact that it uses $N^2$ messages, where $N$ is the number of remaining recipients (since every broadcasting process excludes the original process and itself). Ideally, we’d want to reduce the number of messages required for a reliable broadcast.

**Atomic Broadcast**

Even though the **flooding algorithm** just described can ensure message delivery, it does not guarantee delivery in any particular order. Messages reach their destination eventually, at an unknown time. If we need to deliver messages **in order**, we have to use the **atomic broadcast** (also called the total order multicast), which guarantees both reliable delivery and total order.

While a reliable broadcast ensures that the processes agree on the set of messages delivered, an atomic broadcast also ensures they agree on the same sequence of messages.

In summary, an atomic broadcast has to ensure two essential properties:
* **Atomicity** Processes have to agree on the set of received messages. Either all nonfailed processes deliver the message, or none do.
* **Order** All nonfailed processes deliver the messages in the same order.

Messages here are delivered atomically: every message is either delivered to all processes or none of them and, if the message is delivered, every other message is ordered before or after this message.

**Virtual Synchrony**


* atomic broadcast -> deliver totally ordered messages to a static group of processes
* **virtual synchrony** -> deliver totally ordered messages to a dynamic group of peers

Virtual synchrony organizes processes into **groups**. Each message is uniquely associated with the group it has originated from. As long as the group exists, messages are delivered to all of its members in the same order. In this case, the order is not specified by the model, and some implementations can take this to their advantage for performance gains, as long as the order they provide is consistent across all members.

As soon as one of the participant joins, leaves the group, or fails and is forced out of it, the group view changes. This happens by announcing the group change to all its members. 

Virtual synchrony distinguishes between 
* **message receipt** when a group member receives the message 
* **message delivery** when all the group members receive the message). 

Unless all processes in the group have received it before the view change, no group member can consider this message delivered.

Message was sent in one view should be delivered only in the same view, group views serve as a barrier that message broadcasts cannot pass.


Some **total broadcast algorithms** order messages by using a single process (**sequencer**) that is responsible for determining it. Such algorithms can be easier to implement, but rely on detecting the leader failures for liveness. 

Using a sequencer can improve performance, since we do not need to establish consensus between processes for every message, and can use a sequencer-local view instead. This approach can still scale by partitioning the requests.
Despite its technical soundness, virtual synchrony has not received broad adoption and isn’t commonly used in end-user commercial systems.

**Zookeeper Atomic Broadcast (ZAB)**


One of the most popular and widely known implementations of the atomic broadcast
is **ZAB** used by [**Apache Zookeeper**](https://zookeeper.apache.org/doc/r3.4.13/zookeeperInternals.html) (a hierarchical distributed key-value store, where it’s used to ensure the total order of events and atomic
delivery necessary to maintain consistency between the replica states).


Processes in ZAB can take on one of two roles: 
* **leader** (temporary role)
* **follower** 


To write new records and execute reads that observe the most recent values, clients connect to one of the nodes in the cluster. If the node happens to be a leader, it will handle the request. Otherwise, it forwards the request to the leader.

To guarantee leader uniqueness, the protocol timeline is split into epochs, identified with a unique monotonically- and incrementally-sequenced number. During any epoch, there can be only one leader. 

The process starts from finding a **prospective leader** using any election algorithm, as long as it chooses a process that is up with a high probability. As soon as a prospective leader is established, it executes a protocol in three phases:
* **Discovery** 
The prospective leader learns about the latest epoch known by every other process, and proposes a new epoch that is greater than the current epoch of any follower. Followers respond to the epoch proposal with the identifier of the latest transaction seen in the previous epoch. After this step, no process will accept broadcast proposals for the earlier epochs.
* **Synchronization** 
This phase is used to recover from the previous leader’s failure and bring lagging followers up to speed. The prospective leader sends a message to the followers proposing itself as a leader for the new epoch and collects their acknowledgments. As soon as acknowledgments are received, the leader is established. After this step, followers will not accept attempts to become the epoch leader from any other processes. During synchronization, the new leader ensures that followers have the same history and delivers committed proposals from the established leaders of earlier epochs. These proposals are delivered before any proposal from the new epoch is delivered.
* **Broadcast**
As soon as the followers are back in sync, active messaging starts. During this
phase, the leader receives client messages, establishes their order, and broadcasts them to the followers: it sends a new proposal, waits for a quorum of followers to respond with acknowledgments and, finally, commits it. This process is similar to a **two-phase commit without aborts**: votes are just acknowledgments, and the client cannot vote against a valid leader’s proposal.  Proposals from the leaders from incorrect epochs should not be acknowledged. 

The broadcast phase continues until the leader crashes, is partitioned from the followers, or is suspected to be crashed due to the message delay.

The safety of this protocol is guaranteed if followers ensure they accept proposals only from the leader of the established epoch. Two processes may attempt to get elected, but only one of them can win and establish itself as an epoch leader. It is also assumed that processes perform the prescribed steps in good faith and follow the protocol.


Both the leader and followers rely on **heartbeats** to determine the liveness of the remote processes. If the leader does not receive heartbeats from the quorum of followers, it steps down as a leader, and restarts the election process. Similarly, if one of the followers has determined the leader crashed, it starts a new election process.


Messages are totally ordered, and the leader will not attempt to send the next message until the message that preceded it was acknowledged. Even if some messages are received by a follower more than once, their repeated application do not produce additional side effects, as long as delivery order is followed. 

ZAB is able to handle multiple outstanding concurrent state changes from clients, since a unique leader will receive write requests, establish the event order, and broadcast the changes.

Total message order also allows ZAB to improve recovery efficiency. During the synchronization phase, followers respond with a highest committed proposal. The leader can simply choose the node with the highest proposal for recovery, and this can be the only node messages have to be copied from.

One of the advantages of ZAB is its efficiency: the broadcast process requires only two rounds of messages, and leader failures can be recovered from by streaming the missing messages from a single up-to-date process. Having a long-lived leader can have a positive impact on performance: we do not require additional consensus rounds to establish a history of events, since the leader can sequence them based on its local view.

**Raft**


It was first presented in a paper titled [“In Search of an Understandable Consensus Algorithm”](https://web.stanford.edu/~ouster/cgi-bin/papers/raft-atc14) (implementation **LogCabin**).


Locally, participants store a log containing the sequence of commands executed by the state machine. Since inputs that processes receive are identical and logs contain the same commands in the same order, applying these commands to the state machine guarantees the same output. 

Raft simplifies consensus by making the concept of leader a first-class citizen. A leader is used to coordinate state machine manipulation and replication. 

There are many similarities between Raft and atomic broadcast algorithms, as well as Multi-Paxos: a single leader emerges from replicas, makes atomic decisions, and establishes the message order.


Each participant in Raft can take one of three roles:
* **Candidate**
Leadership is a temporary condition, and any participant can take this role. To
become a leader, the node first has to transition into a candidate state, and
attempt to collect a majority of votes. If a candidate neither wins nor loses the election (the vote is split between multiple candidates and none of them has a majority of votes), the new term is slated and election restarts.
* **Leader**
A current, temporary cluster leader that handles client requests and interacts
with a replicated state machine. The leader is elected for a period called a **term**. Each term is identified by a monotonically increasing number and may continue for an arbitrary time period. A new leader is elected if the current one crashes, becomes unresponsive, or is suspected by other processes to have failed, which can happen because of network partitions and message delays.
* **Follower**
A passive participant that persists log entries and responds to requests from the leader and candidates. Follower in Raft is a role similar to acceptor and learner from Paxos. Every process begins as a follower.


It may happen that different participants disagree on which term is current, since they can find out about the new term at different times, or could have missed the leader election for one or multiple terms. Since each message contains a term identifier, if one of the participants discovers that its term is out-of-date, it updates the term to the higher-numbered one. 

This means that there may be several terms in flight at any given point in time, but the higher-numbered one wins in case of a conflict. A node updates the term only if it starts a new election process or finds out that its term is out-of-date.

On startup, or whenever a follower doesn’t receive messages from the leader and suspects that it has crashed, it starts the leader election process. A participant attempts to become a leader by transitioning into the candidate state and collecting votes from the majority of nodes.

The main components of the Raft algorithm:
* **Leader election**
Candidate P1 sends a RequestVote message to the other processes. This message
includes the candidate’s term, the last term known by it, and the ID of the last log entry it has observed. After collecting a majority of votes, the candidate is successfully elected as a leader for the term. Each process can give its vote to at most one candidate.
* **Periodic heartbeats**
The protocol uses a heartbeat mechanism to ensure the liveness of participants.
The leader periodically sends heartbeats to all followers to maintain its term. If a follower doesn’t receive new heartbeats for a period called an election timeout, it assumes that the leader has failed and starts a new election.
* **Log replication / broadcast**
The leader can repeatedly append new values to the replicated log by sending
AppendEntries messages. The message includes the leader’s term, index, and
term of the log entry that immediately precedes the ones it’s currently sending,
and one or more entries to store.


**Leader Role in Raft**


A leader can be elected **only from the nodes holding all committed entries**: if during the election, the follower’s log information is more up-to-date (in other words, has a higher term ID, or a longer log entry sequence, if terms are equal) than the candidate’s, its vote is denied. To win the vote, a candidate has to collect a majority of votes.

Once elected, the leader has to accept client requests (which can also be forwarded to it from other nodes) and replicate them to the followers. This is done by appending the entry to its log and sending it to all the followers in parallel.

When a follower receives an AppendEntries message, it appends the entries from the message to the local log, and acknowledges the message, letting the leader know that it was persisted. As soon as enough replicas send their acknowledgments, the entry is considered committed and is marked correspondingly in the leader log.


Since only the most up-to-date candidates can become a leader, followers never have to bring the leader up-to-date, and log entries are only flowing from leader to follower and not vice versa.

client | leader | replica 1 | replica 2 | replica 3 
---|---|---|---|---
idle | [x=1, y=7, x=2] | [x=1, y=7, x=2] | [x=1, y=7, x=2] | [x=1, y=7, x=2]
request x = 25 -> | [x=1, y=7, x=2], req: x = 25 | [x=1, y=7, x=2] | [x=1, y=7, x=2] | [x=1, y=7, x=2]
idle | [x=1, y=7, x=2], req: x = 25, notify on req -> | [x=1, y=7, x=2], req: x = 25 | [x=1, y=7, x=2], req: x = 25 | [x=1, y=7, x=2], req: x = 25 
idle | [x=1, y=7, x=2], req: x = 25| <- ack, [x=1, y=7, x=2], req: x = 25 | <- ack, [x=1, y=7, x=2], req: x = 25 | <- ack, [x=1, y=7, x=2], req: x = 25 
idle | [x=1, y=7, x=2, x=25], commit ->| [x=1, y=7, x=2,x=25] | [x=1, y=7, x=2, x=25] | [x=1, y=7, x=2, x=25] 
idle | [x=1, y=7, x=2, x=25] | <- ack, [x=1, y=7, x=2,x=25] | <- ack, [x=1, y=7, x=2, x=25] | <- ack, [x=1, y=7, x=2, x=25] 
req OK | <- ack, [x=1, y=7, x=2, x=25] | [x=1, y=7, x=2,x=25] | [x=1, y=7, x=2, x=25] | [x=1, y=7, x=2, x=25] 

* Leader has the most recent view of the events. The leader proceeds by replicating the entries to the followers, and committing them after collecting acknowledgments. 
* Committing an entry also commits all entries preceding it in the log. 
* Only the leader can make a decision on whether or not the entry can be committed. 
* Each log entry is marked with a term ID and a log index, identifying its position in the log. 
* Committed entries are guaranteed to be replicated to the quorum of participants and are safe to be applied to the state machine in the order they appear in the log

**Failure Scenarios**


When multiple followers decide to become candidates, and no candidate can collect a majority of votes, the situation is called a **split vote**. Raft uses randomized timers to reduce the probability of multiple subsequent elections ending up in a split vote. One of the candidates can start the next election round earlier and collect enough votes, while the others sleep and give way to it. This approach speeds up the election without requiring any additional coordination between candidates.

Followers may be down or slow to respond, and the leader has to make the best effort to ensure message delivery. It can try sending messages again if it doesn’t receive an acknowledgment within the expected time bounds. As a performance optimization, it can send multiple messages in parallel.


**Raft guarantees**


* Since entries replicated by the leader are uniquely identified, repeated message delivery is guaranteed not to break the log order. Followers deduplicate messages using their sequence IDs, ensuring that double delivery has no undesired side effects. Sequence IDs are also used to ensure the log ordering. 

* If entries in two logs on different replicas have the same term and the same index, they store the same command and all entries that precede them are the same.

* Raft guarantees to never show an uncommitted message as a committed one but not vice versa. Already committed messages can still be seen as in progress, which is a rather harmless property and can be worked around by retrying a client command until it is finally committed.

* For failure detection, the leader has to send heartbeats to the followers. This way, the leader maintains its term. When one of the nodes notices that the current leader is down, it attempts to initiate the election. 

* The newly elected leader has to restore the state of the cluster to the last known up-to-date log entry. 
 - It does so by finding a **common ground** (the highest log entry on which both the leader and follower agree), 
 - ordering followers to discard all (uncommitted) entries appended after this point
 - then sends the most recent entries from its log, overwriting the followers’ history. 
 
* The leader’s own log records are never removed or overwritten: it can only append entries to its own log.

* Only one leader can be elected at a time for a given term; no two leaders can be active during the same term.

* Committed log entries are guaranteed to be present in logs for subsequent leaders and cannot get reverted, since before the entry is committed it is known to be replicated by the leader.

* All messages are identified uniquely by the message and term IDs; neither current nor subsequent leaders can reuse the same identifier for the different entry.

Since its appearance, Raft has become very popular and is currently used in many
databases and other distributed systems, including CockroachDB, Etcd, and Consul. This can be attributed to its simplicity, but also may mean that Raft lives up to the promise of being a reliable consensus algorithm.

**Byzantine Consensus**


All the consensus algorithms we have been discussing so far assume non-Byzantine
failures. In other words, nodes execute the algorithm in “good faith” and do not try to exploit it or forge the results.


However, distributed systems are sometimes deployed in potentially adversarial environments where the nodes are not controlled by the same entity, and we need algorithms that can ensure a system can function correctly even if some nodes behave erratically or even maliciously. Besides ill intentions, Byzantine failures can also be caused by bugs, misconfiguration, hardware issues, or data corruption.

Most Byzantine consensus algorithms require $N^2$ messages to complete an algorithm step, where $N$ is the size of the quorum, since each node in the quorum has to communicate with each other. This is required to cross-validate each step against other nodes, since nodes cannot rely on each other or on the leader and have to verify other nodes’ behaviors by comparing returned results with the majority responses.


* We suppose all communication between the nodes is encrypted, which serves to prevent message forging and network attacks. 
* Replicas know one another’s public keys to verify identities and encrypt messages. 
* **Note** Faulty nodes may leak information from inside the system, since, even though encryption is used, every node needs to interpret message contents to react upon them.

**PBFT Algorithm**


* PBFT assumes independent node failures (i.e., failures can be coordinated, but the entire system cannot be taken over at once, or at least with the same exploit method). 
* The system makes weak synchrony assumptions, like how you would expect a network to behave normally: failures may occur, but they are not indefinite and are eventually recovered from.

For PBFT to guarantee both safety and liveness, no more than (n - 1)/3 replicas can be faulty (where n is the total number of participants). For a system to sustain f compromised nodes, it is required to have at least n = 3f + 1 nodes. This is the case because a majority of nodes have to agree on the value: f replicas might be faulty, and there might be f replicas that are not responding but may not be faulty (for example, due to a network partition, power failure, or maintenance). The algorithm has to be able to collect enough responses from nonfaulty replicas to still outnumber those from the faulty ones.


* Consensus properties: all nonfaulty replicas have to agree both on the set of received values and their order, despite the possible failures.

* To distinguish between cluster configurations, PBFT uses views. In each view, one of the replicas is a primary and the rest of them are considered backups. 

* All nodes are numbered consecutively, and the index of the primary node is v mod N, where v is the view ID, and N is the number of nodes in the current configuration. The view can change in cases when the primary fails. 

* Clients execute their operations against the primary. The primary broadcasts the requests to the backups, which execute the requests and send a response back to the client. The client waits for f + 1 replicas to respond with the same result for any operation to succeed.

After the primary receives a client request, protocol execution proceeds in three phases:
* **Pre-prepare** 
The primary broadcasts a message containing a view ID, a unique monotonically increasing identifier, a payload (client request), and a **payload digest**. Digests are computed using a strong collision-resistant hash function, and are **signed** by the sender. The backup accepts the message if its view matches with the primary view and the client request hasn’t been tampered with: the calculated payload digest matches the received one. 
* **Prepare** 
If the backup accepts the pre-prepare message, it enters the prepare phase and
starts broadcasting Prepare messages, containing a view ID, message ID, and a
payload digest, but without the payload itself, to all other replicas (including the primary). Replicas can move past the prepare state **only** if they receive 2f prepares from different backups that match the message received during preprepare: they have to have the same view, same ID, and a digest.
* **Commit**
After that, the backup moves to the commit phase, where it broadcasts Commit
messages to all other replicas and waits to collect 2f + 1 matching Commit messages (possibly including its own) from the other participants. A digest in this case is used to reduce the message size during the prepare phase, since
it’s not necessary to rebroadcast an entire payload for verification, as the digest serves as a payload summary. 




During the prepare and commit phases, nodes communicate by sending messages to
each other node and waiting for the messages from the corresponding number of
other nodes, to check if they match and make sure that incorrect messages are not broadcasted. Peers cross-validate all messages so that only nonfaulty nodes can successfully commit messages. If a sufficient number of matching messages cannot be collected, the node doesn’t move to the next step.

When replicas collect enough commit messages, they notify the client, finishing the round. The client cannot be certain about whether or not execution was fulfilled correctly until it receives f + 1 matching responses.


View changes occur when replicas notice that the primary is inactive, and suspect that it might have failed. Nodes that detect a primary failure stop responding to further messages (apart from checkpoint and view-change related ones), broadcast a view change notification, and wait for confirmations. When the primary of the new view receives 2f view change events, it initiates a new view.

To reduce the number of messages in the protocol, clients can collect 2f + 1 matching responses from nodes that tentatively execute a request (e.g., after they’ve collected a sufficient number of matching Prepared messages). If the client cannot collect enough matching tentative responses, it retries and waits for f + 1 nontentative responses as described previously.

Read-only operations in PBFT can be done in just one round-trip. The client sends a read request to all replicas. Replicas execute the request in their tentative states, after all ongoing state changes to the read value are committed, and respond to the client. After collecting 2f + 1 responses with the same value from different replicas, the operation completes.

**Recovery and Checkpointing**


Replicas save accepted messages in a stable log. Every message has to be kept until it has been executed by at least f + 1 nodes. This log can be used to get other replicas up to speed in case of a network partition, but recovering replicas need some means of verifying that the state they receive is correct, since otherwise recovery can be used as an attack vector.

To show that the state is correct, nodes compute a digest of the state for messages up to a given sequence number. Nodes can compare digests, verify state integrity, and make sure that messages they received during recovery add up to a correct final state.

This process is too expensive to perform on every request. 
* After every N requests, where N is a configurable constant, the primary makes a stable checkpoint, where it broadcasts the latest sequence number of the latest request whose execution is reflected in the state, and the digest of this state. 
* It then waits for 2f + 1 replicas to respond. These responses constitute a proof for this checkpoint, and a guarantee that replicas can safely discard state for all pre-prepare, prepare, commit, and checkpoint messages up to the given sequence number.

Byzantine fault tolerance is essential to understand and is used in storage systems deployed in potentially adversarial networks. Most of the time, it is enough to authenticate and encrypt internode communication, but when there’s no trust between the parts of the system, algorithms similar to PBFT have to be employed.

Since algorithms resistant to Byzantine faults impose significant overhead in terms of the number of exchanged messages, it is important to understand their use cases.